In [ ]:
import h5py
import matplotlib.pyplot as plt
import numpy as np

from neuropacks import NHP
from pyuoi.decomposition import UoI_CUR, CUR
from scipy.io import loadmat
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

%matplotlib inline

In [ ]:
nhp = NHP(data_path='/Users/psachdeva/data/NHP/indy_20160411_01.mat')

In [ ]:
Y = nhp.get_response_matrix(
    bin_width=0.25, region='M1'
)
cursor_pos = nhp.get_binned_positions(bin_width=0.25)

In [ ]:
max_ks = np.arange(25, 150, 5)
n_max_ks = max_ks.size
n_splits = 10
uoi_columns_dict = {}
cur_columns_dict = {}
uoi_decoding = np.zeros(n_max_ks)
cur_decoding = np.zeros(n_max_ks)
uoi_scores_x = np.zeros((n_max_ks, n_splits))
uoi_scores_y = np.zeros(uoi_scores_x.shape)
cur_scores_x = np.zeros(uoi_scores_x.shape)
cur_scores_y = np.zeros(uoi_scores_x.shape)

In [ ]:
for k_idx, max_k in enumerate(max_ks):
    uoi_cur = UoI_CUR(
        n_boots=20,
        max_k=max_k,
        boots_frac=0.8
    )
    uoi_cur.fit(Y)

    uoi_columns = uoi_cur.columns_
    uoi_columns_dict[k_idx] = uoi_columns
    
    n_columns = uoi_columns.size
    
    cur = CUR(max_k=max_k)
    cur.fit(Y, c=n_columns)
    cur_columns = np.sort(cur.columns_[:n_columns])
    cur_columns_dict[k_idx] = cur_columns
    
    Y_uoi = Y[:, uoi_columns]
    Y_cur = Y[:, cur_columns]
    
    Y_uoi_err = Y - np.dot(Y_uoi, np.dot(np.linalg.pinv(Y_uoi), Y))
    Y_cur_err = Y - np.dot(Y_cur, np.dot(np.linalg.pinv(Y_cur), Y))
    uoi_decoding[k_idx] = np.sum(np.abs(Y_uoi_err))/Y.size
    cur_decoding[k_idx] = np.sum(np.abs(Y_cur_err))/Y.size

    kf = KFold(n_splits=10)
    for fold_idx, (train_idx, test_idx) in enumerate(kf.split(Y)):
        Y_train, pos_train_x, pos_train_y = Y[train_idx], cursor_pos[train_idx, 0], cursor_pos[train_idx, 1]
        Y_test, pos_test_x, pos_test_y = Y[test_idx], cursor_pos[test_idx, 0], cursor_pos[test_idx, 1]

        # decode with uoi columns
        ols = LinearRegression()
        ols.fit(Y_train[:, uoi_columns], pos_train_x)
        uoi_scores_x[k_idx, fold_idx] = ols.score(Y_test[:, uoi_columns], pos_test_x)
        
        ols = LinearRegression()
        ols.fit(Y_train[:, uoi_columns], pos_train_y)
        uoi_scores_y[k_idx, fold_idx] = ols.score(Y_test[:, uoi_columns], pos_test_y)

        # decode with cur columns
        ols = LinearRegression()
        ols.fit(Y_train[:, cur_columns], pos_train_x)
        cur_scores_x[k_idx, fold_idx] = ols.score(Y_test[:, cur_columns], pos_test_x)
        
        ols = LinearRegression()
        ols.fit(Y_train[:, cur_columns], pos_train_y)
        cur_scores_y[k_idx, fold_idx] = ols.score(Y_test[:, cur_columns], pos_test_y)

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(8, 6), sharex=True)

axes[0].plot(max_ks, np.median(uoi_scores_x, axis=1), color='r', linewidth=3)
axes[0].plot(max_ks, np.median(cur_scores_x, axis=1), color='k', linewidth=3)

axes[1].plot(max_ks, np.median(uoi_scores_y, axis=1), color='r', linewidth=3, label=r'\textbf{UoI}$_{\textbf{CSS}}$')
axes[1].plot(max_ks, np.median(cur_scores_y, axis=1), color='k', linewidth=3, label=r'\textbf{CSS}')

axes[1].set_xlabel(r'$k_{\text{max}}$', fontsize=25)

fig.text(
    x=-0.025, y=0.55, s=r'\textbf{Explained}' '\n' r'\textbf{Variance}',
    rotation=90,
    horizontalalignment='center',
    verticalalignment='center',
    fontsize=23
)
axes[0].set_ylabel(r'$x$ position', fontsize=23)
axes[1].set_ylabel(r'$y$ position', fontsize=23)

axes[1].set_xlim([25, 145])
axes[1].set_xticks([25, 50, 75, 100, 125, 145])

axes[1].legend(loc=4, prop={'size': 20})

axes[0].set_title(r'\textbf{Hand Position Decoding Accuracy}', fontsize=25)
plt.tight_layout()

plt.savefig('m1_cur_decoding_position.pdf', bbox_inches='tight')

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(8, 6), sharex=True)
axes[0].plot(max_ks, uoi_decoding, color='r', linewidth=3, label=r'\textbf{UoI}$_{\textbf{CSS}}$')
axes[0].plot(max_ks, cur_decoding, color='k', linewidth=3, label=r'\textbf{CSS}')

axes[0].set_xlim([25, 145])
axes[0].set_xticks([25, 50, 75, 100, 125, 145])

axes[0].set_ylabel(r'\textbf{Error}', fontsize=22)
axes[0].set_title(r'\textbf{Reconstruction Error of Data Matrix}', fontsize=23)
axes[0].legend(loc=0, prop={'size': 20})


axes[1].plot(
    max_ks, [uoi_columns_dict[key].size for key in uoi_columns_dict.keys()],
    color='k',
    linewidth=3)

axes[1].set_xlim([25, 145])
axes[1].set_xticks([25, 50, 75, 100, 125, 145])

axes[1].set_title(r'\textbf{Number of Columns Selected}', fontsize=23)
axes[1].set_xlabel(r'$k_{\text{max}}$', fontsize=25)
axes[1].set_ylabel(r'\textbf{Num. Columns}', fontsize=22)
                   
plt.tight_layout()
plt.savefig('m1_cur_error.pdf', bbox_inches='tight')

In [ ]:
results = h5py.File('cur_results.h5', 'w')
results['uoi_decoding_x'] = uoi_scores_x
results['uoi_decoding_y'] = uoi_scores_y
results['cur_decoding_x'] = cur_scores_x
results['cur_decoding_y'] = cur_scores_y
results['uoi_reconstruction'] = uoi_decoding
results['cur_reconstruction'] = cur_decoding
results.attrs['file'] = 'indy_20160411_01.mat'
results.attrs['bin_size'] = 0.25
for idx, column in uoi_columns_dict.items():
    results['uoi_columns/' + str(max_ks[idx])] = column
    
for idx, column in cur_columns_dict.items():
    results['cur_columns/' + str(max_ks[idx])] = column
results.close()